# Seq2Seq

本文介绍以下知识点：
- Mapping one text sequence to another with a neural network
- Understanding sequence-to-sequence tasks and how they’re different from the others you’ve learned about
- Using encoder-decoder model architectures for translation and chat
- Training a model to pay attention to what is important in a sequence

seq2seq model 的输入和输出可以是不同的length.


## 1. Encoder-Decoder Architecture

- **sequence encoder**:
    - input: a sequence (e.g., text)
    - output: lower dimension representation (e.g., thought vector)

这个我们在前面的LSTM 网络结构中已经见过了。

- **sequence decoder**:
    - input: vector
    - output: a sequence (e.g., text)
    
这个有些类似我们之前用LSTM 网络生成文字的，但是也有不同，因为我们不能让它随机输出，而是要输出特定的内容。

encoder 部分，LSTM 把语义encode 成一个thought vector (也称作**context vector**). 这个thought vector 就是我们的**information bottleneck**。 相当于把一句话的意思压缩成一个n-dimensional vector. 

然后这个thought vector 作为decoder network 的input. 如下图所示。

<img src="img/thought_vector.png" alt="drawing" width="500"/>

这个thought vector 包含两部分：
- memory state `state_c`
- output of hidden layer `state_h`

然后我们设置`return_state=True`，LSTM model 会输出hidden layer state.

decoder network 使用thought vector 作为输入的初始状态，然后需要一个**start token**, 然后就可以generate 后续的tokens 了。
如下图所示。

<img src="img/next_word_prediction.png" alt="drawing" width="500"/>

#### training stage

- starting text: input to encoder
- expected text: input to decoder

#### inference stage

1. 2 networks: encoder and decoder
2. encoder: generate thought vector
3. decoder: 接受thought vector 作为initial state，使用一个start token 和initial state，生成第一个token (的word vector)。
4. 然后类似与generative network，逐步预测下一个token，更新state，预测再下一个，一直到生成了一个stop token，或者token 个数到达上线

### 1.1 对比other techniques

#### Auto-Encoder

这和auto-encoder 很像，auto-encoder 也是在某个中间层找到一个低维的**bottle of information**，使用了encoder-decoder architecture.

区别在于，auto-encoder 的目标时寻找一个dense vector representation（相当于数据压缩），目标是能够尽可能地还原原数据。也就是输入和输出时相同的。而seq2seq 的输入和输出是不同的seq，但是具有相同的meaning.

- **variational auto-encoder**: dense vector 服从高斯分布

#### PCA
PCA 的目标是降维，通常降至2维或3维。而且不是面向变长的time-series data.


### 1.2 translation v.s. chatbot

translation 和chatbot 都需要把一个sequence map 到另一个sequence。但是区别在于，chatbot 需要学习更加复杂的mapping。
- 需要更多的数据: enough Q&A pairs
- 更高维度的thought vector ———— 需要encode all the information about the world.

### 1.3 Challenges

#### 挑战1: 输入和输出的长度不同

例如，我们需要把英文翻译成德语，可能输入的长度和输出的长度不同。

<img src="img/translation.png" alt="drawing" width="500"/>

#### 挑战2: More control on generative model

之前的LSTM 生成token 的过程如下图所示：

<img src="img/unrolled_encoder_decoder.png" alt="drawing" width="700"/>



## 2. Assembling a sequence-to-sequence pipeline

下面，我们使用keras 来构建一个seq2seq network. 

### 2.1 Preprocessing

#### padding

pad to fix length, 通常，是match the longest input sequence. 注意，input 和target 的sequence length 可以不同，但是padding 后的每个input 和target length 必须相同。如下图所示。

<img src="img/before_preprocessing.png" alt="drawing" width="700"/>

#### start / stop annotation

除此以外，target sequence should be annotated with the start and stop tokens, 用来告诉decoder when the job starts and when it is done. 如下图所示。

<img src="img/after_preprocessing.png" alt="drawing" width="700"/>

记住，我们需要两个版本的target sequences:
- 第一个版本带有start / stop tag, 用于作为decoder network 的输入
- 第二个版本不带有tag，用于loss function 作为accuracy score 的计算

传统的training set 都是一个tuple:
- feature vactor: x
- label: y

seq2seq training set的每个traininig example 是一个triplet:
- initial input
- expected output with tag
- expected output without tag

### 2.2 Sequence encoder

**目的**：create thought vector.

注意，encoder 不能直接训练，因为没有labeled thought vector，所以没法计算损失函数。所以encoder 一般要和decoder 一起训练。

我们丢弃network 每一步的output，指标流final state。

<img src="img/thought_encoder.png" alt="drawing" width="500"/>



### 2.3 Thought decoder

- 使用thought vector + tagged expected output 作为输入
- 使用output 作为输出

<img src="img/thought_decoder.png" alt="drawing" width="500"/>

